# Trino / Iceberg connector data management demo sample
Copyright (C) 2021 OS-Climate

This sample shows:
* How Arrow and Parquet can be used to provide direct support for persisting dataFrames and metadata together in highly portable and performant files. This mechanism allows us to manage data ingestion and metadata in a transactional and integrated way.
* Apache Iceberg ACID transaction and time travel capabilities used for data set versioning


Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
# pip install boto3
# pip install python-dotenv
# pip install trino
# pip install pandas
# pip install pyarrow

In [2]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import boto3
import pandas as pd
import uuid
import pyarrow as pa
import pyarrow.parquet as pq
import json
import trino
import io

Load Environment Variables

In [3]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Create a simple data frame for testing

In [4]:
# initialise data of lists of powerplants and capacity
data = {'name':['Albas', 'Albi', 'Albias', 'Allaire'],
        'capacity_mw':[1.8, 6.70448, 2.41, 8.2]}
# Create DataFrame
df = pd.DataFrame(data)
# Add a unique identifier to the data set
uid = str(uuid.uuid4())
df['uuid'] = uid
# Print the output
df

,name,capacity_mw,uuid
0,Albas,1.80000,56dc0472-318e-47a9-9b49-a51e111cd7c7
1,Albi,6.70448,56dc0472-318e-47a9-9b49-a51e111cd7c7
2,Albias,2.41000,56dc0472-318e-47a9-9b49-a51e111cd7c7
3,Allaire,8.20000,56dc0472-318e-47a9-9b49-a51e111cd7c7


Convert the DataFrame to an Arrow table using PyArrow and inspect th table’s metadata property 

In [5]:
table = pa.Table.from_pandas(df)
print(table.schema.metadata)

{b'pandas': b'{"index_columns": [{"kind": "range", "name": null, "start": 0, "stop": 4, "step": 1}], "column_indexes": [{"name": null, "field_name": null, "pandas_type": "unicode", "numpy_type": "object", "metadata": {"encoding": "UTF-8"}}], "columns": [{"name": "name", "field_name": "name", "pandas_type": "unicode", "numpy_type": "object", "metadata": null}, {"name": "capacity_mw", "field_name": "capacity_mw", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "uuid", "field_name": "uuid", "pandas_type": "unicode", "numpy_type": "object", "metadata": null}], "creator": {"library": "pyarrow", "version": "5.0.0"}, "pandas_version": "1.3.3"}'}


Create custom meta data and key

In [6]:
custom_meta_content = {
    'title': 'Global Power Plant Database',
    'description': 'A comprehensive, global, open source database of power plants',
    'version': '1.3.0',
    'release_date': '20210602'
}
custom_meta_key = 'metaset'

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [7]:
custom_meta_json = json.dumps(custom_meta_content)
existing_meta = table.schema.metadata
combined_meta = {
    custom_meta_key.encode(): custom_meta_json.encode(),
    **existing_meta
}

Replace the original table metadata by the combined metadata in the Arrow table. This table now contains both the custom metadata and the Pandas metadata. 

In [8]:
table = table.replace_schema_metadata(combined_meta)
print(table.schema.metadata)

{b'metaset': b'{"title": "Global Power Plant Database", "description": "A comprehensive, global, open source database of power plants", "version": "1.3.0", "release_date": "20210602"}', b'pandas': b'{"index_columns": [{"kind": "range", "name": null, "start": 0, "stop": 4, "step": 1}], "column_indexes": [{"name": null, "field_name": null, "pandas_type": "unicode", "numpy_type": "object", "metadata": {"encoding": "UTF-8"}}], "columns": [{"name": "name", "field_name": "name", "pandas_type": "unicode", "numpy_type": "object", "metadata": null}, {"name": "capacity_mw", "field_name": "capacity_mw", "pandas_type": "float64", "numpy_type": "float64", "metadata": null}, {"name": "uuid", "field_name": "uuid", "pandas_type": "unicode", "numpy_type": "object", "metadata": null}], "creator": {"library": "pyarrow", "version": "5.0.0"}, "pandas_version": "1.3.3"}'}


Open an S3 client connection and save the dataframe as a parquet file so that dataFrame & metadata are now coupled together

In [9]:
# Create an S3 client
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [10]:
tablename = 'gppd'
pq.write_table(
    table,
    '/tmp/{tname}.parquet'.format(tname=tablename)
)
# df.to_parquet('/tmp/{tname}.parquet'.format(tname=tablename), index=False)
s3.upload_file(
    Bucket=os.environ['S3_DEV_BUCKET'],
    Key='trino/wri/{tname}/{tname}.parquet'.format(tname=tablename),
    Filename='/tmp/{tname}.parquet'.format(tname=tablename)
)

Restore data and metadata

In [11]:
# Read the Parquet file into an Arrow table
obj = s3.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key='trino/wri/{tname}/{tname}.parquet'.format(tname=tablename)
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)

In [12]:
restored_df

,name,capacity_mw,uuid
0,Albas,1.80000,56dc0472-318e-47a9-9b49-a51e111cd7c7
1,Albi,6.70448,56dc0472-318e-47a9-9b49-a51e111cd7c7
2,Albias,2.41000,56dc0472-318e-47a9-9b49-a51e111cd7c7
3,Allaire,8.20000,56dc0472-318e-47a9-9b49-a51e111cd7c7


In [13]:
restored_meta

{'title': 'Global Power Plant Database',
 'description': 'A comprehensive, global, open source database of power plants',
 'version': '1.3.0',
 'release_date': '20210602'}

Open a Trino connection using JWT for authentication

In [14]:
conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

In [15]:
# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

[['aicoe_osc_demo'],
 ['company_data'],
 ['default'],
 ['defaultschema1'],
 ['demo'],
 ['eje'],
 ['epacems'],
 ['epacems_y95_al'],
 ['information_schema'],
 ['metastore'],
 ['pudl'],
 ['rmi_utility_transition_hub'],
 ['team1'],
 ['team2'],
 ['testaccessschema1'],
 ['urgentem'],
 ['wri'],
 ['wri_gppd']]

Build a map and define schema mapping logic for parquet to sql

In [16]:
_p2smap = {
    'object': 'varchar',
    'string': 'varchar',
    'float32': 'real',
    'Float32': 'real',
    'float64': 'double',
    'Float64': 'double',
    'int32': 'integer',
    'Int32': 'integer',
    'int64': 'bigint',
    'Int64': 'bigint',
    'category': 'varchar',
    'datetime64[ns, UTC]': 'timestamp',
    'datetime64[ns]': 'timestamp'
}

def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))

# add ability to specify optional dict for specific fields?
# if column name is present, use specified value?
def generate_table_schema_pairs(df):
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0],t=e[1]) for e in pz])

Create ingestion schema based on source data name and remove old tables if necessary

In [17]:
cur.execute('create schema if not exists osc_datacommons_dev.wri')
cur.fetchall()

[[True]]

In [18]:
cur.execute('drop table if exists osc_datacommons_dev.wri.' + tablename)
cur.fetchall()

[[True]]

In [19]:
schema = generate_table_schema_pairs(df)
tabledef = """create table if not exists osc_datacommons_dev.wri.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/wri/{tname}/'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],tname=tablename)
print(tabledef)

# tables created externally may not show up immediately in cloud-beaver
cur.execute(tabledef)
cur.fetchall()

create table if not exists osc_datacommons_dev.wri.gppd(
    name varchar,
    capacity_mw double,
    uuid varchar
) with (
    format = 'parquet',
    external_location = 's3a://ocp-odh-os-demo-s3/trino/wri/gppd/'
)


[[True]]

In [27]:
# Create metastore structure
metastore = {'catalog':'osc_datacommons_dev',
             'schema':'wri',
             'table':tablename,
             'uuid':uid,
             'metadata':custom_meta_content}
# Create DataFrame
df_meta = pd.DataFrame(metastore)
# Print the output
df_meta

,catalog,schema,table,uuid,metadata
description,osc_datacommons_dev,wri,gppd,56dc0472-318e-47a9-9b49-a51e111cd7c7,"A comprehensive, global, open source database ..."
release_date,osc_datacommons_dev,wri,gppd,56dc0472-318e-47a9-9b49-a51e111cd7c7,20210602
title,osc_datacommons_dev,wri,gppd,56dc0472-318e-47a9-9b49-a51e111cd7c7,Global Power Plant Database
version,osc_datacommons_dev,wri,gppd,56dc0472-318e-47a9-9b49-a51e111cd7c7,1.3.0
